In [5]:
#TODOS:
#* vectorized environment*
#* dynamic multi-processing based on execution time
#* atomic functions
#* checkpoints with branching, pick best if better than current
#* save progression for visualization in readme
#* tensorboard hyperparameter tuning

In [6]:
# Auxiliar imports
import sys, os
import matplotlib.pyplot as plt
import numpy as np

# Gym imports
import gym
from gym.vector import SyncVectorEnv

# PyTorch imports
import torch
from torch import nn, optim

# Custom imports
sys.path.append(os.path.abspath('..')) # Add parent directory to path
import ppo_network, importlib
importlib.reload(ppo_network) # Prevents caching issues with notebooks
from ppo_network import PPONetwork